## Stadtentwicklung

Open in Colab: [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/febse/opt2025-de/blob/main/IP-Urban-Planning.ipynb)

Angenommen, Sofia steht vor einem lokalen Haushaltsdefizit und der Stadtrat überlegt, wie die Grundsteuereinnahmen durch die Sanierung städtischer Grundstücke erhöht werden können. Das Projekt besteht aus zwei Teilen: der Beseitigung vernachlässigter und beschädigter Gebäude und dem Bau neuer Wohnungen.

1. Die Gemeinde besitzt derzeit 300 baufällige Gebäude, die abgerissen werden könnten. Jedes dieser Gebäude macht 1,000 Quadratmeter frei und der Abriss kostet 3,000 EUR pro Gebäude. 15 Prozent der befreiten Fläche sind für Straßen, Gehwege und Freiflächen vorgesehen.
2. Auf den geräumten Grundstücken kann die Gemeinde vier Arten von neuen Wohngebäuden errichten: Einfamilienhäuser (300 Quadratmeter), Zweifamilienhäuser (500 Quadratmeter), Dreifamilienhäuser (700 Quadratmeter) und Vierfamilienhäuser (900 Quadratmeter). Die geschätzten Steuereinnahmen belaufen sich auf EUR 1,000, EUR 1,700,  EUR 2,400 bzw.  EUR 2,800 pro Jahr.
3. Mindestens 20 Prozent der Neubauten müssen Einfamilienhäuser sein, Zweifamilienhäuser müssen mindestens 20 Prozent ausmachen, und Drei- und Vierfamilienhäuser müssen (zusammen) mindestens ein Viertel aller Neubauten ausmachen.
4. Die Baukosten für neue Häuser betragen 50,000 EUR, 70,000 EUR, 130,000 EUR bzw. 160,000 EUR.
5. Die Gemeinde beabsichtigt, das Projekt durch ein Bankdarlehen zu finanzieren, das 15 Millionen EUR nicht überschreiten darf.

Wie viele Häuser jedes Typs sollte die Gemeinde bauen, um die höchstmöglichen Steuereinnahmen zu erzielen?

Der Datensatz `homes` enthält die Daten für die vier Arten von Wohngebäuden:

- `cost`: Die Baukosten für ein Haus dieses Typs.
- `tax`: Die geschätzten Steuereinnahmen pro Jahr für ein Haus dieses Typs.
- `area`: Die Fläche, die ein Haus dieses Typs einnimmt.

:::{.callout-note collapse="true"}
## Das mathematische Modell

Die Entscheidungsvariablen

$$
\begin{align*}
x_{1} & : \text{Anzahl von Einfamilienhäusern} \\
x_{2} & : \text{Anzahl von Zweifamilienhäusern} \\
x_{3} & : \text{Anzahl von Dreifamilienhäusern} \\
x_{4} & : \text{Anzahl von Vierfamilienhäusern} \\
x_{d} & : \text{Anzahl der abgerissenen Gebäude}
\end{align*}
$$

Die Zielfunktion (in 1000 EUR)

$$
\max x_{1} + 1.7x_{2} + 2.4x_{3} + 2.8x_{4}
$$

Einschränkungen

1. Flächenbeschränkung

$$
300x_{1} + 500x_{2} + 700x_{3} + 900x_{4} \leq (1 - 0.15) \cdot 1000 \cdot x_{d}
$$

2. Anzahl der abgerissenen Gebäude

$$
x_{d} \leq 300
$$

3. Mindestanforderungen

$$
x_{all} = x_{1} + x_{2} + x_{3} + x_{4}
$$

$$
\begin{align*}
x_{1} \geq 0.2 x_{all} \\
x_{2} \geq 0.2 x_{all} \\
x_{3} + x_{4} \geq 0.25 x_{all}
\end{align*}
$$

1. Budgetbeschränkung (in 1000 EUR)

$$
50x_{1} + 70x_{2} + 130x_{3} + 160x_{4} + 3x_{d} \leq 15000
$$

:::


In [1]:
%pip install gurobipy

import pandas as pd
import gurobipy as gp
from gurobipy import GRB

homes = pd.DataFrame({
    'type': ['single', 'double', 'triple', 'quad'],
    'cost': [50000, 70000, 130000, 160000],
    "tax": [1000, 1700, 2400, 2800],
    'area': [300, 500, 700, 900],
}).set_index('type')

/home/amarov/stats/opt2026-de/.venv/bin/python3: No module named pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Unser bisherige Ansatz für die Umsetzung des Modells war es, die Entscheidungsvariablen
# einzeln zu definieren.

model1 = gp.Model('homes-1')
model1.Params.LogToConsole = 0

x_1 = model1.addVar(vtype=GRB.INTEGER, name='x_1')
x_2 = model1.addVar(vtype=GRB.INTEGER, name='x_2')
x_3 = model1.addVar(vtype=GRB.INTEGER, name='x_3')
x_4 = model1.addVar(vtype=GRB.INTEGER, name='x_4')
x_d = model1.addVar(vtype=GRB.INTEGER, name='x_d')

# Zielfunktion

model1.setObjective(1000 * x_1 + 1700 * x_2 + 2400 * x_3 + 2800 * x_4, GRB.MAXIMIZE) 

# Nebenbedingungen

model1.addConstr(300 * x_1 + 500 * x_2 + 700 * x_3 + 900 * x_4 <= 1000 * (1 - 0.15) * x_d, 'area')
model1.addConstr(50 * x_1 + 70 * x_2 + 130 * x_3 + 160 * x_4 + 3 * x_d <= 15000, 'budget')
model1.addConstr(x_d <= 300, 'demolished')

x_all = x_1 + x_2 + x_3 + x_4

model1.addConstr(x_1 >= 0.2 * x_all, 'single')
model1.addConstr(x_2 >= 0.2 * x_all, 'double')
model1.addConstr(x_3 + x_4 >= 0.25 * x_all, 'triple and quad')

model1.optimize()

# Das Modell als LP-Datei speichern und anzeigen
model1.write('homes-1.lp')

with open('homes-1.lp', 'r') as f:
    print(f.read())

# Die Lösung des Modells as DataFrame speichern

vars_df = pd.DataFrame.from_records([(v.varName, v.x) for v in model1.getVars()], columns=['variable', 'value'])
vars_df.set_index('variable', inplace=True)
vars_df

Restricted license - for non-production use only - expires 2027-11-29


Set parameter LogToConsole to value 0


\ Model homes-1
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  1000 x_1 + 1700 x_2 + 2400 x_3 + 2800 x_4
Subject To
 area: 300 x_1 + 500 x_2 + 700 x_3 + 900 x_4 - 850 x_d <= 0
 budget: 50 x_1 + 70 x_2 + 130 x_3 + 160 x_4 + 3 x_d <= 15000
 demolished: x_d <= 300
 single: 0.8 x_1 - 0.2 x_2 - 0.2 x_3 - 0.2 x_4 >= 0
 double: - 0.2 x_1 + 0.8 x_2 - 0.2 x_3 - 0.2 x_4 >= 0
 triple_and_quad: - 0.25 x_1 - 0.25 x_2 + 0.75 x_3 + 0.75 x_4 >= 0
Bounds
Generals
 x_1 x_2 x_3 x_4 x_d
End



,value
variable,
x_1,36.0
x_2,99.0
x_3,42.0
x_4,3.0
x_d,109.0


Nun möchten wir das Modell implementieren, ohne die einzelnen Variablen per Hand zu erstellen. Der Datensatz `homes` enthält die Daten für die vier Arten von Wohngebäuden:

In [3]:
homes

,cost,tax,area
type,,,
single,50000,1000,300
double,70000,1700,500
triple,130000,2400,700
quad,160000,2800,900


In [4]:
# Der Index des DataFrames `homes` ist der Typ der Häuser
homes.index

Index(['single', 'double', 'triple', 'quad'], dtype='str', name='type')

In [5]:
model2 = gp.Model('Beispiel')

# Erstellt eine Variable für jeden Wert des Index (für jeden Haustyp in diesem Fall)
x = model2.addVars(homes.index, vtype=GRB.INTEGER, name='x')
model2.update()

x

{'single': <gurobi.Var x[single]>,
 'double': <gurobi.Var x[double]>,
 'triple': <gurobi.Var x[triple]>,
 'quad': <gurobi.Var x[quad]>}

In [6]:
model2.setObjective(x.prod(homes['tax'].to_dict()), GRB.MAXIMIZE)

model2.write('homes-2.lp')

with open('homes-2.lp', 'r') as f:
    print(f.read())


\ Model Beispiel
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  1000 x[single] + 1700 x[double] + 2400 x[triple] + 2800 x[quad]
Subject To
Bounds
Generals
 x[single] x[double] x[triple] x[quad]
End



In [7]:
# Zugriff auf eine Variable
x['single']

<gurobi.Var x[single]>

In [8]:
x['quad']

<gurobi.Var x[quad]>

In [9]:
# Eine Linearkombination von zwei Variablen

2 * x['single'] + 3 * x['quad']

<gurobi.LinExpr: 2.0 x[single] + 3.0 x[quad]>

In [10]:
# Die Summe aller Variablen

x.sum()

<gurobi.LinExpr: x[single] + x[double] + x[triple] + x[quad]>

In [11]:
# Die Werte einer Spalte des DataFrames als Dictionary. Die 
# Werte des Index dienen als Schlüssel.

homes['area'].to_dict()

{'single': 300, 'double': 500, 'triple': 700, 'quad': 900}

In [12]:
# Man kann auf die Werte des Dictionary über die Schlüssel zugreifen

homes['area'].to_dict()['single']

300

In [13]:
# Eine Linearkombination mit Koeffizienten aus einem Dictionary

x.prod(homes['area'].to_dict())

<gurobi.LinExpr: 300.0 x[single] + 500.0 x[double] + 700.0 x[triple] + 900.0 x[quad]>

In [14]:
# Eine lineare Ungleichung mit Koeffizienten aus einem Dictionary

x.prod(homes['area'].to_dict()) <= 2000

<gurobi.TempConstr: 300.0 x[single] + 500.0 x[double] + 700.0 x[triple] + 900.0 x[quad] <= 2000>

In [15]:
m = gp.Model('homes')
m.Params.LogToConsole = 0

x = m.addVars(homes.index, vtype=GRB.INTEGER, name='x')
x_d = m.addVar(vtype=GRB.INTEGER, name='demolished')

# Die Zielfunktion

m.setObjective(x.prod(homes['tax'].to_dict()), GRB.MAXIMIZE)

# Die Einschränkungen

## Anzahl der abgerissenen Häuser

m.addConstr(x_d <= 300, 'demolished')

## Kosten
m.addConstr(x.prod(homes['cost'].to_dict()) + 3000 * x_d <= 15e6, 'costs')

## Fläche
m.addConstr(x.prod(homes['area'].to_dict()) <= 1000 * (1 - 0.15) * x_d, 'area')

## Anzahl der Einfamilienhäuser
m.addConstr(x['single'] >= 0.2 * x.sum(), 'single')

## Anzahl der Zweifamilienhäuser
m.addConstr(x['double'] >= 0.2 * x.sum(), 'double')

## Anzahl der Drei- und Vierfamilienhäuser
m.addConstr(x['triple'] + x['quad'] >= 0.25 * x.sum(), 'triple and quad homes')

# Die Lösung

m.optimize()

m.write('homes.lp')

with open('homes.lp', 'r') as f:
    print(f.read())

Set parameter LogToConsole to value 0


\ Model homes
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  1000 x[single] + 1700 x[double] + 2400 x[triple] + 2800 x[quad]
Subject To
 demolished: demolished <= 300
 costs: 50000 x[single] + 70000 x[double] + 130000 x[triple]
   + 160000 x[quad] + 3000 demolished <= 1.5e+07
 area: 300 x[single] + 500 x[double] + 700 x[triple] + 900 x[quad]
   - 850 demolished <= 0
 single: 0.8 x[single] - 0.2 x[double] - 0.2 x[triple] - 0.2 x[quad] >= 0
 double: - 0.2 x[single] + 0.8 x[double] - 0.2 x[triple] - 0.2 x[quad]
   >= 0
 triple_and_quad_homes: - 0.25 x[single] - 0.25 x[double] + 0.75 x[triple]
   + 0.75 x[quad] >= 0
Bounds
Generals
 x[single] x[double] x[triple] x[quad] demolished
End



In [16]:
# Die Lösung als DataFrame

vars_df = pd.DataFrame(
    [(v.varName, v.x) for v in m.getVars()],
    columns=['variable', 'value']
)

vars_df

,variable,value
0,x[single],36.0
1,x[double],99.0
2,x[triple],42.0
3,x[quad],3.0
4,demolished,109.0


In [17]:
# Die Einschränkungen as DataFrame

constrs_df = pd.DataFrame(
    [(c.constrName, c.slack) for c in m.getConstrs()],
    columns=['constraint', 'slack']
)

constrs_df

,constraint,slack
0,demolished,1.910000e+02
1,costs,3.000000e+03
2,area,2.500000e+02
3,single,4.440892e-16
4,double,-6.300000e+01
5,triple and quad homes,0.000000e+00
